In [1]:
from torch.utils.data import DataLoader
import torch.nn as nn

from trading import *

In [2]:
loss = nn.L1Loss()

In [5]:
NIFTY = YTicker("^NSEI")
data = NIFTY.history(period=periods.max, interval=intervals.day)[["High", "Low", "Close", "Prev Close"]]
data

,High,Low,Close,Prev Close
Date,,,,
2007-09-17 00:00:00+05:30,4549.049805,4482.850098,4494.649902,NaN
2007-09-18 00:00:00+05:30,4551.799805,4481.549805,4546.200195,4494.649902
2007-09-19 00:00:00+05:30,4739.000000,4550.250000,4732.350098,4546.200195
2007-09-20 00:00:00+05:30,4760.850098,4721.149902,4747.549805,4732.350098
2007-09-21 00:00:00+05:30,4855.700195,4733.700195,4837.549805,4747.549805
...,...,...,...,...
2024-06-03 00:00:00+05:30,23338.699219,23062.300781,23263.900391,22530.699219
2024-06-04 00:00:00+05:30,23179.500000,21281.449219,21884.500000,23263.900391
2024-06-05 00:00:00+05:30,22670.400391,21791.949219,22620.349609,21884.500000


In [3]:
dataset = models.TickerUpDownDataset(data, lookahead=1)
dataloader = DataLoader(dataset, batch_size=len(dataset))

model = models.UpClassifier()
tester = models.Backtest(model, dataloader, loss)
tester.test()


Test Error:
    Accuracy: 53.3%
    Avg loss: 0.467415
    Correct: 2182/4097


In [4]:
data["RSI"] = discretize_bin(RSIIndicator()(data))
data["SMA"] = discretize_bool(SMAIndicator(window=12)(data) > SMAIndicator(window=30)(data))
data["AO"] = discretize_bin(AwesomeOscillatorIndicator()(data))
data["Up"] = discretize_bool(data["Close"] > data["Prev Close"])

del data["High"], data["Low"], data["Prev Close"]
data.dropna(inplace=True)

In [5]:
dataset = models.TickerUpDownDataset(data, lookahead=1)
dataloader = DataLoader(dataset, batch_size=len(dataset))

model = models.BakedMarkovClassifier(dataset)
tester = models.Backtest(model, dataloader, loss)
tester.test()


Test Error:
    Accuracy: 56.2%
    Avg loss: 0.438484
    Correct: 2282/4064


In [23]:
train = data.sample(frac=0.5)
test = data.drop(train.index).reset_index(drop=True)

train_dataset = models.TickerUpDownDataset(train, lookahead=1)
test_dataset = models.TickerUpDownDataset(test, lookahead=1)

model = models.BakedMarkovClassifier(train_dataset)
dataloader = DataLoader(test_dataset, batch_size=len(test))

tester = models.Backtest(model, dataloader, loss)
tester.test()


Test Error:
    Accuracy: 55.3%
    Avg loss: 0.447343
    Correct: 1123/2032
